In [1]:
import sys
sys.path.append("../src/")
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
# create sparksession
spark = SparkSession \
    .builder \
    .appName("Pysparkexample") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
from config import *

In [2]:
pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [3]:
df = spark.read.parquet("../data/census_income.parquet")

In [4]:
df.limit(5).toPandas()

,age,class_worker,det_ind_code,det_occ_code,education,wage_per_hour,hs_college,marital_stat,major_ind_code,major_occ_code,race,hisp_origin,sex,union_member,unemp_reason,full_or_part_emp,capital_gains,capital_losses,stock_dividends,tax_filer_stat,region_prev_res,state_prev_res,det_hh_fam_stat,det_hh_summ,mig_chg_msa,mig_chg_reg,mig_move_reg,mig_same,mig_prev_sunbelt,num_emp,fam_under_18,country_father,country_mother,country_self,citizenship,own_or_self,vet_question,vet_benefits,weeks_worked,year,income_50k
0,58,Private,37.0,38.0,10th grade,0.0,Not in universe,Divorced,Business and repair services,Transportation and material moving,White,All other,Male,Not in universe,Not in universe,Full-time schedules,0.0,0.0,24.0,Single,Not in universe,Not in universe,Nonfamily householder,Householder,?,?,?,Not in universe under 1 year old,?,2.0,Not in universe,?,United-States,United-States,Native- Born in the United States,0.0,Not in universe,2.0,52.0,95.0,- 50000.
1,14,Not in universe,0.0,0.0,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,Black,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0.0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,0.0,0.0,94.0,- 50000.
2,9,Not in universe,0.0,0.0,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0.0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,0.0,0.0,94.0,- 50000.
3,66,Not in universe,0.0,0.0,High school graduate,0.0,Not in universe,Divorced,Not in universe or children,Not in universe,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Single,Not in universe,Not in universe,Nonfamily householder,Householder,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0.0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,2.0,0.0,94.0,- 50000.
4,21,Private,32.0,42.0,High school graduate,0.0,Not in universe,Married-civilian spouse present,Wholesale trade,Handlers equip cleaners etc,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Joint both under 65,West,Vermont,Spouse of householder,Spouse of householder,Not identifiable,Different state same division,Different state in West,No,No,5.0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,2.0,Not in universe,2.0,49.0,94.0,- 50000.


In [5]:
from pyspark.ml.feature import ChiSqSelector, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline

from pyspark.ml.feature import QuantileDiscretizer

In [7]:
for col in ['age', 'wage_per_hour', 'capital_gains', 'capital_losses', 'stock_dividends', 'weeks_worked']:
    df = QuantileDiscretizer(numBuckets=10, inputCol=col,outputCol=col + "_q").fit(df).transform(df)
    df = df.drop(col)

In [8]:
df.limit(5).toPandas()

,class_worker,det_ind_code,det_occ_code,education,hs_college,marital_stat,major_ind_code,major_occ_code,race,hisp_origin,sex,union_member,unemp_reason,full_or_part_emp,tax_filer_stat,region_prev_res,state_prev_res,det_hh_fam_stat,det_hh_summ,mig_chg_msa,mig_chg_reg,mig_move_reg,mig_same,mig_prev_sunbelt,num_emp,fam_under_18,country_father,country_mother,country_self,citizenship,own_or_self,vet_question,vet_benefits,year,income_50k,age_q,wage_per_hour_q,capital_gains_q,capital_losses_q,stock_dividends_q,weeks_worked_q
0,Private,37.0,38.0,10th grade,Not in universe,Divorced,Business and repair services,Transportation and material moving,White,All other,Male,Not in universe,Not in universe,Full-time schedules,Single,Not in universe,Not in universe,Nonfamily householder,Householder,?,?,?,Not in universe under 1 year old,?,2.0,Not in universe,?,United-States,United-States,Native- Born in the United States,0.0,Not in universe,2.0,95.0,- 50000.,8.0,1.0,1.0,1.0,2.0,4.0
1,Not in universe,0.0,0.0,Children,Not in universe,Never married,Not in universe or children,Not in universe,Black,All other,Female,Not in universe,Not in universe,Children or Armed Forces,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0.0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,0.0,94.0,- 50000.,2.0,1.0,1.0,1.0,1.0,1.0
2,Not in universe,0.0,0.0,Children,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0.0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,0.0,94.0,- 50000.,1.0,1.0,1.0,1.0,1.0,1.0
3,Not in universe,0.0,0.0,High school graduate,Not in universe,Divorced,Not in universe or children,Not in universe,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,Single,Not in universe,Not in universe,Nonfamily householder,Householder,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0.0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,2.0,94.0,- 50000.,8.0,1.0,1.0,1.0,1.0,1.0
4,Private,32.0,42.0,High school graduate,Not in universe,Married-civilian spouse present,Wholesale trade,Handlers equip cleaners etc,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,Joint both under 65,West,Vermont,Spouse of householder,Spouse of householder,Not identifiable,Different state same division,Different state in West,No,No,5.0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,2.0,Not in universe,2.0,94.0,- 50000.,3.0,1.0,1.0,1.0,1.0,3.0


In [9]:
string_cols = [col for col, dtype in df.dtypes if dtype=="string"]

In [10]:
indexers = [StringIndexer(inputCol=col, outputCol=col+"_ind").fit(df) for col in string_cols]

In [11]:
pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)

In [12]:
for col in string_cols:
    df = df.drop(col)

In [13]:
df.limit(5).toPandas()

,det_ind_code,det_occ_code,num_emp,own_or_self,vet_benefits,year,age_q,wage_per_hour_q,capital_gains_q,capital_losses_q,stock_dividends_q,weeks_worked_q,class_worker_ind,education_ind,hs_college_ind,marital_stat_ind,major_ind_code_ind,major_occ_code_ind,race_ind,hisp_origin_ind,sex_ind,union_member_ind,unemp_reason_ind,full_or_part_emp_ind,tax_filer_stat_ind,region_prev_res_ind,state_prev_res_ind,det_hh_fam_stat_ind,det_hh_summ_ind,mig_chg_msa_ind,mig_chg_reg_ind,mig_move_reg_ind,mig_same_ind,mig_prev_sunbelt_ind,fam_under_18_ind,country_father_ind,country_mother_ind,country_self_ind,citizenship_ind,vet_question_ind,income_50k_ind
0,37.0,38.0,2.0,0.0,2.0,95.0,8.0,1.0,1.0,1.0,2.0,4.0,1.0,5.0,0.0,2.0,7.0,9.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,94.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,94.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,94.0,8.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,32.0,42.0,5.0,2.0,2.0,94.0,3.0,1.0,1.0,1.0,1.0,3.0,1.0,0.0,0.0,1.0,13.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,32.0,2.0,2.0,8.0,6.0,6.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df.dtypes

[('det_ind_code', 'double'),
 ('det_occ_code', 'double'),
 ('num_emp', 'double'),
 ('own_or_self', 'double'),
 ('vet_benefits', 'double'),
 ('year', 'double'),
 ('age_q', 'double'),
 ('wage_per_hour_q', 'double'),
 ('capital_gains_q', 'double'),
 ('capital_losses_q', 'double'),
 ('stock_dividends_q', 'double'),
 ('weeks_worked_q', 'double'),
 ('class_worker_ind', 'double'),
 ('education_ind', 'double'),
 ('hs_college_ind', 'double'),
 ('marital_stat_ind', 'double'),
 ('major_ind_code_ind', 'double'),
 ('major_occ_code_ind', 'double'),
 ('race_ind', 'double'),
 ('hisp_origin_ind', 'double'),
 ('sex_ind', 'double'),
 ('union_member_ind', 'double'),
 ('unemp_reason_ind', 'double'),
 ('full_or_part_emp_ind', 'double'),
 ('tax_filer_stat_ind', 'double'),
 ('region_prev_res_ind', 'double'),
 ('state_prev_res_ind', 'double'),
 ('det_hh_fam_stat_ind', 'double'),
 ('det_hh_summ_ind', 'double'),
 ('mig_chg_msa_ind', 'double'),
 ('mig_chg_reg_ind', 'double'),
 ('mig_move_reg_ind', 'double'),
 ('m

In [15]:
# put features into a feature vector column
assembler = VectorAssembler(inputCols=[col for col in df.columns if col != "income_50k_ind"], 
                            outputCol="features") 

In [16]:
df = assembler.transform(df)

In [17]:
df.limit(5).toPandas()

,det_ind_code,det_occ_code,num_emp,own_or_self,vet_benefits,year,age_q,wage_per_hour_q,capital_gains_q,capital_losses_q,stock_dividends_q,weeks_worked_q,class_worker_ind,education_ind,hs_college_ind,marital_stat_ind,major_ind_code_ind,major_occ_code_ind,race_ind,hisp_origin_ind,sex_ind,union_member_ind,unemp_reason_ind,full_or_part_emp_ind,tax_filer_stat_ind,region_prev_res_ind,state_prev_res_ind,det_hh_fam_stat_ind,det_hh_summ_ind,mig_chg_msa_ind,mig_chg_reg_ind,mig_move_reg_ind,mig_same_ind,mig_prev_sunbelt_ind,fam_under_18_ind,country_father_ind,country_mother_ind,country_self_ind,citizenship_ind,vet_question_ind,income_50k_ind,features
0,37.0,38.0,2.0,0.0,2.0,95.0,8.0,1.0,1.0,1.0,2.0,4.0,1.0,5.0,0.0,2.0,7.0,9.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,"(37.0, 38.0, 2.0, 0.0, 2.0, 95.0, 8.0, 1.0, 1...."
1,0.0,0.0,0.0,0.0,0.0,94.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 94.0, 2.0, 1.0, 1.0,..."
2,0.0,0.0,0.0,0.0,0.0,94.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 94.0, 1.0, 1.0, 1.0,..."
3,0.0,0.0,0.0,0.0,2.0,94.0,8.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(0.0, 0.0, 0.0, 0.0, 2.0, 94.0, 8.0, 1.0, 1.0,..."
4,32.0,42.0,5.0,2.0,2.0,94.0,3.0,1.0,1.0,1.0,1.0,3.0,1.0,0.0,0.0,1.0,13.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,32.0,2.0,2.0,8.0,6.0,6.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[32.0, 42.0, 5.0, 2.0, 2.0, 94.0, 3.0, 1.0, 1...."


In [18]:
selector = ChiSqSelector(featuresCol="features", selectorType="fdr",
                         outputCol="selectedFeatures", labelCol="income_50k_ind")

df = selector.fit(df).transform(df)

In [19]:
result = df.limit(5).toPandas()

In [20]:
result.selectedFeatures.map(lambda x: len(x))

0    37
1    37
2    37
3    37
4    37
Name: selectedFeatures, dtype: int64